# Predicting Donation Projects Outcome Based on DonorsChoose.org Data

## Extract, Transform, Load (ETL) Notebook

#### *Author: Kunyu He*
#### *University of Chicago CAPP'20*

### Executive Summary

In this ETL notebook, I load the data, which comes separately in `.csv` format from [IBM Cloud Object Storage](https://www.ibm.com/cloud/object-storage?S_PKG=AW&cm_mmc=Search_Google-_-Cloud_Cloud+Platform-_-WW_NA-_-+ibm++object++storage_Broad_&cm_mmca1=000016GC&cm_mmca2=10007090&cm_mmca7=9060146&cm_mmca8=aud-311016886972:kwd-346458796492&cm_mmca9=_k_CjwKCAiAyfvhBRBsEiwAe2t_i-XCqy6aVw7VL5rPgPbazlACBDB8tL5qFioP_k0oLEF8dxisH8cTlBoClHoQAvD_BwE_k_&cm_mmca10=317209285867&cm_mmca11=b&mkwid=_k_CjwKCAiAyfvhBRBsEiwAe2t_i-XCqy6aVw7VL5rPgPbazlACBDB8tL5qFioP_k0oLEF8dxisH8cTlBoClHoQAvD_BwE_k_|1445|530530&cvosrc=ppc.google.%2Bibm%20%2Bobject%20%2Bstorage&cvo_campaign=000016GC&cvo_crid=317209285867&Matchtype=b&gclid=CjwKCAiAyfvhBRBsEiwAe2t_i-XCqy6aVw7VL5rPgPbazlACBDB8tL5qFioP_k0oLEF8dxisH8cTlBoClHoQAvD_BwE), as Pandas DataFrames and performs data preprocessing with Python's [Pandas](https://pandas.pydata.org/) library. The data cleaning part includes the following:

* Extracting relevant columns from the `outcomes` and `resources` data sets
* Merging all separate dataframes on primary key `projectid`
* Extracting the original training set based on dates the projects were posted *(only those posted before 01/01/2014)* and split it into new training and test set
* Dropping irrelevant columns and columns with more than 10% values missing
* Encoding some columns as binary dummies and converting some others to categoricals
* Dropping rows that still contain NA values

After preprocessing, the output dataframes *(`train` and `test`)* are stored in `.csv` format back to object storage as data assets named accordingly *(`train.csv` and `test.csv`)* for future use.

### Extracting Data

Data used for this project comes from Kaggle competition [KDD Cup 2014 - Predicting Excitement at DonorsChoose.org](https://www.kaggle.com/c/kdd-cup-2014-predicting-excitement-at-donors-choose/overview). Kaggle users who participate in the competition could get access to data in CSV format [here](https://www.kaggle.com/c/3926/download-all).

Basically there are five data sets, namely `projects`, `resources`, `essays`, `donations` and `outcomes`:

* `projects` - contains information about each project. This is provided for both the training and test set.
* `resources` - contains information about the resources requested for each project. This is provided for both the training and test set.
* `essays` - contains project text posted by the teachers. This is provided for both the training and test set.
* `donations` - contains information about the donations to each project. This is only provided for projects in the training set.
* `outcomes` - contains information about the outcomes of projects in the training set.

**Since `outcomes` is only provided for the training set and access to test set is blocked by Kaggle. Information on whether a project posted after 2014-01-01 got fully funded is not available. Hence we only use the original training set and split it into new training and test set.**

**Notice that once we know the total price a project required, combined with total donations the project got, predicting `fully_funded` would be meaningless. So we won't use `donations` data set to train or test our model.**

### Loading Data

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

Use the chunks below to list the data assets in my IBM Cloud Object Storage linked to this project. As it includes my credentials, the code is hidden from unauthorized viewers.

In [2]:
# The code was removed by Watson Studio for sharing.

Load data into the environment.

Note that information in other columns from the `outcomes` data set can be used to infer whether a project got fully funded with certainty, I'm only using outcome of the donation projects *(`fully_funded`)* and whether the project has a comment thread with greater than average unique comments *(`great_chat`)* from the `outcomes` data set.

Also note that I'm only using the number of items required and statistics of `item_unit_price` as potential predictors from the `resources` data set. As I regard other columns as irrelevant.

In [3]:
projects = pd.read_csv(project.get_file('Donation-Projects-Outcome-Prediction.data.projects.csv'))
outcomes = pd.read_csv(project.get_file('Donation-Projects-Outcome-Prediction.data.outcomes.csv'),
                       usecols=['projectid', 'fully_funded', 'great_chat'])
resources = pd.read_csv(project.get_file('Donation-Projects-Outcome-Prediction.data.resources.csv'),
                        usecols=['projectid', 'item_unit_price'])
essays = pd.read_csv(project.get_file('Donation-Projects-Outcome-Prediction.data.essays.csv'))

print("Names of loaded dataframes: ", [instance for instance in dir() if isinstance(eval(instance),
                                                                                    pd.core.frame.DataFrame)])

Names of loaded dataframes:  ['essays', 'outcomes', 'projects', 'resources']


### Transforming Data

**We are only using the number of items required and statistics of `item_unit_price` as potential predictors from the `resources` data set.**

In [4]:
resources = resources.groupby('projectid')['item_unit_price'].agg(['count', 'min', 'median', 'max']).reset_index()
resources.columns = ['projectid', 'number_of_items', 'min_price_items', 'median_price_items', 'max_price_items']

**Merge `projects`, `outcomes`, `essays` and `resources` data on `projectid`.**

In [5]:
merged = projects.merge(outcomes, how='outer', on='projectid').merge(essays, how='outer', on='projectid').merge( \
                        resources, how='outer', on='projectid')
merged.date_posted = pd.to_datetime(merged.date_posted)

**Extract the original training set from the merged data set according to dates the projects were posted. Split it into new training set and test set with a test proportion of 0.25.**

In [6]:
train, test = train_test_split(merged[merged.date_posted < "2014-01-01"], test_size=0.25, random_state=999)
print("There are {} observations in training set (75% of the observations), {} in test set and {} features.".format( \
      train.shape[0], test.shape[0], train.shape[1]))

There are 464494 observations in training set (75% of the observations), 154832 in test set and 46 features.


Define a function for data cleaning and perform the cleaning on both train and test set.

In [7]:
def clean(df):
    # drop features with 10% or more values missing
    df.drop(df.columns[df.isnull().sum(axis=0) > df.projectid.nunique()*0.1], axis=1, inplace=True)
    
    # redundant information to drop
    df.drop(['projectid', 'teacher_acctid_x', 'schoolid', 'school_ncesid', 'school_latitude',
             'school_longitude', 'school_city', 'school_district', 'school_county', 'school_zip',
             'teacher_acctid_y', 'fulfillment_labor_materials'], axis=1, inplace=True)
    
    # convert to binary dummy
    binary = ['school_charter', 'school_magnet', 'school_year_round', 'school_nlns',
              'school_kipp', 'school_charter_ready_promise', 'teacher_teach_for_america',
              'teacher_ny_teaching_fellow', 'eligible_double_your_impact_match',
              'eligible_almost_home_match', 'fully_funded', 'great_chat']
    for col in binary:
        df[col].replace(to_replace=["f", "t"], value=[0, 1], inplace=True)
        df[col] = df[col].astype("category")

    # convert to categorical
    cat = ['school_state', 'teacher_prefix', 'primary_focus_subject', 'primary_focus_area',
           'resource_type', 'poverty_level', 'grade_level']
    for col in cat:
        df[col] = df[col].astype("category")

    return df.dropna(axis=0, inplace=False)

In [8]:
train = clean(train)
test = clean(test)

print("Now there are {} observations in the training set, {} in the test set and {} features.".format( \
      train.shape[0], test.shape[0], train.shape[1]))

Now there are 458051 observations in the training set, 152697 in the test set and 31 features. Data is ready for analysis


### Data Outputs Storage

Now save both training set and test set as `.csv` files and upload them back to my IBM Cloud Object Storage bucket.

In [9]:
project.save_data(data=train.to_csv(index=True),
                  file_name='Donation-Projects-Outcome-Prediction.data.train.csv', overwrite=True)

{'asset_id': 'f5450447-79fb-4b53-b646-5fbb9a220a8f',
 'bucket_name': 'donationprojectsoutcomeprediction-donotdelete-pr-felyzh04iugf9l',
 'file_name': 'Donation-Projects-Outcome-Prediction.data.train.csv',
 'message': 'File Donation-Projects-Outcome-Prediction.data.train.csv has been written successfully to the associated OS'}

In [10]:
project.save_data(data=test.to_csv(index=True),
                  file_name='Donation-Projects-Outcome-Prediction.data.test.csv', overwrite=True)

{'asset_id': '7a2d8b2c-65c5-4258-8605-b95653bd30c5',
 'bucket_name': 'donationprojectsoutcomeprediction-donotdelete-pr-felyzh04iugf9l',
 'file_name': 'Donation-Projects-Outcome-Prediction.data.test.csv',
 'message': 'File Donation-Projects-Outcome-Prediction.data.test.csv has been written successfully to the associated OS'}

Check whether the output data files are successfully uploaded.

In [11]:
project.get_files()

[{'asset_id': '7a2d8b2c-65c5-4258-8605-b95653bd30c5',
  'name': 'Donation-Projects-Outcome-Prediction.data.test.csv'},
 {'asset_id': 'f5450447-79fb-4b53-b646-5fbb9a220a8f',
  'name': 'Donation-Projects-Outcome-Prediction.data.train.csv'},
 {'asset_id': 'b6920713-693a-454e-855f-a24c95efd8ce',
  'name': 'Donation-Projects-Outcome-Prediction.data.projects.csv'},
 {'asset_id': '9b1c8961-1564-41a7-8402-89fded8d7e21',
  'name': 'Donation-Projects-Outcome-Prediction.data.outcomes.csv'},
 {'asset_id': '42bb6e53-4a3c-48f5-bcfb-5a7ab6b4461c',
  'name': 'Donation-Projects-Outcome-Prediction.data.resources.csv'},
 {'asset_id': '260878e4-a8c3-4c74-8070-48813214e8b2',
  'name': 'Donation-Projects-Outcome-Prediction.data.essays.csv'}]

Now that our data is stored properly, the ETL process for this project is done.

**Cheers!**